<a href="https://colab.research.google.com/github/Julialunna/Artificial-Intelligence/blob/main/DP-PSO-SGD/DP_breast_cancer_PSO_SGD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.optim as optim
import copy
import random
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from pydp.algorithms.numerical_mechanisms import GaussianMechanism

In [1]:
!pip install python-dp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 24.7 MB/s eta 0:00:00


In [3]:
breast_cancer = load_breast_cancer()

x=breast_cancer.data
y=breast_cancer.target

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)  # Padroniza os dados de treino
x_test = scaler.transform(x_test)  # Padroniza os dados de teste (usando os mesmos parâmetros do treino)

# Converter para tensores do PyTorch
x_train= torch.tensor(x_train, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)



In [4]:
class MLP(torch.nn.Module):
  def __init__(self, input_size, output_size):
    #chama superclasse
    super(MLP, self).__init__()
    self.fc1 = torch.nn.Linear(input_size, 128)
    self.fc2 = torch.nn.Linear(128, 64)
    self.fc3 = torch.nn.Linear(64, 32)
    self.fc4 = torch.nn.Linear(32, 16)
    self.fc5 = torch.nn.Linear(16, output_size)
    #define o comportamento da rede neural
  def forward(self, x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = torch.relu(self.fc3(x))
    x = torch.relu(self.fc4(x))
    x = self.fc5(x)
    return x


In [5]:
class Particle:
    def __init__(self, model, device):
        self.model = copy.deepcopy(model).to(device)
        self.best_model = copy.deepcopy(model).to(device)
        # self.position = {name: torch.zeros_like(param).to(device) for name, param in model.named_parameters()}
        # self.velocity = {name: torch.zeros_like(param).to(device) for name, param in model.named_parameters()}

        # Definir os limites do espaço de busca e a escala da velocidade
        #DIMINUIR LOW E HIGH
        low = -10.0  # Limite inferior do espaço de busca
        high = 10.0  # Limite superior do espaço de busca
        velocity_scale = 0.1  # Escala para as velocidades iniciais

        # Inicializar a posição com valores aleatórios uniformes no intervalo [low, high]
        self.position = {name: torch.rand_like(param).to(device) * (high - low) + low for name, param in model.named_parameters()}

        # Inicializar a velocidade com valores aleatórios pequenos (normalmente distribuídos)
        self.velocity = {name: torch.randn_like(param).to(device) * velocity_scale for name, param in model.named_parameters()}

        self.best_score = float('inf')
        self.device = device

        # Inicializar o otimizador (por exemplo, Adam)
        #self.optimizer = optim.Adam(self.model.parameters(), lr=0.0001)
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.0001, weight_decay=1e-5)

    def pso(self, global_best_model, inertia, c1, c2):
        for name, param in self.model.named_parameters():
            # if param.grad is None:
            #     continue

            local_rand = random.random()
            global_rand = random.random()

            # Atualização da velocidade
            self.velocity[name] = (
                inertia*self.velocity[name]
                + c1*local_rand*(self.best_model.state_dict()[name].to(self.device) - param.data)
                + c2*global_rand*(global_best_model.state_dict()[name].to(self.device) - param.data)
            )

            # Atualizar posição
            self.position[name] = param.data + self.velocity[name]
            param.data = self.position[name]

    def evaluate_weights(self, x, y, criterion):
        self.model.eval()
        total_loss = 0
        correct = 0
        total = len(x_test)

        with torch.no_grad():

          outputs = self.model(x_test)
          loss = criterion(outputs, y_test)
          total_loss += loss.item()

          predictions = torch.sigmoid(outputs)
          predictions = (predictions > 0.5).float()

          correct = (predictions == y_test).sum().item()


        accuracy = correct / total
        avg_loss = total_loss / total
        return avg_loss, accuracy * 100

In [6]:
# Definir os hiperparâmetros do PSO e do Adam
pop_size = 10
num_epochs = 150
#inertia = 0.9
c1, c2 = 0.8, 0.9
learning_rate = 0.008
beta1, beta2 = 0.9, 0.999
epsilon = 1e-8

In [8]:
model = MLP(input_size=x_train.size()[1], output_size=1)

particles = [Particle(model, device) for _ in range(pop_size)]

global_best_model = copy.deepcopy(particles[0].model)
global_best_score = float('inf')

criterion = torch.nn.BCEWithLogitsLoss()

overall_global_best_accuracy = 0.0
overall_global_best_model = copy.deepcopy(global_best_model)

# Loop de treinamento do PSO
for epoch in range(num_epochs):
    inertia = 0.9 - ((0.9-0.4)/num_epochs)*epoch
    for particle in particles:
        # Colocar o modelo em modo de treinamento
        particle.model.train()

        particle.optimizer.zero_grad()

        # Treinar a partícula (atualização de posição)
        particle.pso(global_best_model, inertia, c1, c2)

        outputs = particle.model(x_train)
        loss = criterion(outputs, y_train)
        loss.backward()

        epsilon = 5
        delta = 1e-5
        sensitivity = 2

        sigma = sensitivity * torch.sqrt((2.0 * torch.log(torch.tensor(1.0 / delta))).clone().detach()) / epsilon

        # Percorrer todos os parâmetros do modelo e adicionar ruído aos gradientes
        torch.nn.utils.clip_grad_norm_(particle.model.parameters(), max_norm=1.0)
        for param in particle.model.parameters():
            if param.grad is not None:
                noise = torch.normal(mean=0, std=sigma, size=param.grad.shape, device=param.grad.device)
                param.grad += noise  # 🔹 Adiciona ruído diretamente ao gradiente

        particle.optimizer.step()

        val_loss, val_acc = particle.evaluate_weights(x_train, y_train, criterion)

        # Avaliar a partícula e atualizar o local best

        if val_loss < particle.best_score:
            particle.best_score = val_loss
            particle.best_model = copy.deepcopy(particle.model)

    #Determinar e atualizar o g-best (modelo global)
    best_particle = min(particles, key=lambda p: p.best_score)
    if best_particle.best_score < global_best_score:
        global_best_score = best_particle.best_score
        global_best_model = copy.deepcopy(best_particle.best_model)
    val_loss, val_accuracy = best_particle.evaluate_weights(x_test, y_test, criterion)
    if val_accuracy > overall_global_best_accuracy:
        overall_global_best_accuracy = val_accuracy
        overall_global_best_model = copy.deepcopy(best_particle.best_model)

    # Avaliar e imprimir a cada época
    if (epoch + 1) % 10 == 0:

        print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.2f}, Validation Accuracy: {val_accuracy:.2f}')
print(f'Best Models accuracy: {overall_global_best_accuracy:.2f}%')

Epoch 10/150, Validation Loss: 0.00, Validation Accuracy: 92.98
Epoch 20/150, Validation Loss: 0.01, Validation Accuracy: 62.28
Epoch 30/150, Validation Loss: 0.00, Validation Accuracy: 93.86
Epoch 40/150, Validation Loss: 0.00, Validation Accuracy: 95.61
Epoch 50/150, Validation Loss: 0.00, Validation Accuracy: 96.49
Epoch 60/150, Validation Loss: 0.00, Validation Accuracy: 97.37
Epoch 70/150, Validation Loss: 0.00, Validation Accuracy: 98.25
Epoch 80/150, Validation Loss: 0.00, Validation Accuracy: 98.25
Epoch 90/150, Validation Loss: 0.00, Validation Accuracy: 98.25
Epoch 100/150, Validation Loss: 0.00, Validation Accuracy: 98.25
Epoch 110/150, Validation Loss: 0.00, Validation Accuracy: 98.25
Epoch 120/150, Validation Loss: 0.00, Validation Accuracy: 98.25
Epoch 130/150, Validation Loss: 0.00, Validation Accuracy: 98.25
Epoch 140/150, Validation Loss: 0.00, Validation Accuracy: 98.25
Epoch 150/150, Validation Loss: 0.00, Validation Accuracy: 98.25
Best Models accuracy: 99.12%
